In [8]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [9]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer_size = 30
        self.lstm = nn.LSTM(input_size=20, hidden_size=self.hidden_layer_size, num_layers=2, dropout=0.5)   
        self.linear1 = nn.Linear(self.hidden_layer_size, self.hidden_layer_size)  # equivalent to Dense in keras
        self.linear2 = nn.Linear(self.hidden_layer_size, 1)  # equivalent to Dense in keras
        self.hidden_cell = (torch.zeros(2, 1, self.hidden_layer_size),
                            torch.zeros(2, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        prediction = self.linear2(self.linear1(lstm_out.view(len(input_seq), -1)))
        return prediction

In [10]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [11]:
data = np.genfromtxt("ML_Datasets/processed_data_regression.csv", delimiter=",")
X = data[:, :-1]
y = data[:, -1]

# normalize the data
X = MinMaxScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

In [13]:
epochs = 100

for i in range(epochs):

    X_train, y_train = shuffle(X_train, y_train)

    optimizer.zero_grad()
    model.hidden_cell = (torch.zeros(2, 1, model.hidden_layer_size),
                         torch.zeros(2, 1, model.hidden_layer_size))

    y_preds = model(X_train)

    train_loss = loss_function(y_preds.view(-1), y_train)
    train_loss.backward()
    optimizer.step()

    if i % 10 == 1:
        print(f'epoch: {i:3} loss: {train_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {train_loss.item():10.8f}')

epoch:   1 loss: 2144.33227539
epoch:  11 loss: 1844.83227539
epoch:  21 loss: 1160.10339355
epoch:  31 loss: 604.97161865
epoch:  41 loss: 580.27032471
epoch:  51 loss: 553.26641846
epoch:  61 loss: 543.92120361
epoch:  71 loss: 543.15985107
epoch:  81 loss: 541.19268799
epoch:  91 loss: 541.09881592
epoch:  99 loss: 540.83337402


In [15]:
model.eval()
    
with torch.no_grad():

    y_test_preds = torch.empty(len(y_test))

    for j in range(len(X_test)):
        y_test_preds[j] = model(X_test[j, :].view(1, len(X_test[j, :])))

    test_loss = loss_function(y_test_preds, y_test)

print(f'Test Loss: {test_loss.item():10.10f}')

Test Loss: 542.1914062500
